In [1]:
import pandas as pd

In [2]:
def getTop(df, column, n=10):
    l = df[column].to_dict().items()
    return sorted(l, key=lambda x : x[-1], reverse=True)[:n]

def getRank(df, column):
        return {k:i for i, (k, v) in enumerate(getTop(df, column, n=None), 1)}

In [3]:
data = pd.read_csv('./all_features.csv')
data = data.set_index('uid')

data['text_mood'] = data['text_mood'].fillna(value=0.0)
data[['is_nsfw', 'has_smile', 'number_of_faces', 'has_face']] = \
data[['is_nsfw', 'has_smile', 'number_of_faces', 'has_face']].fillna(0.0).astype(int)

data.to_csv('.all_features.csv', index_label='vk_id')

In [4]:
data.columns

Index(['gender', 'followers_qty', 'friends_qty', 'pagerank_score',
       'eigencentrality_score', 'hits_in', 'hits_out', 'flw_hits_in',
       'flw_hits_out', 'is_nsfw', 'has_smile', 'number_of_faces', 'has_face',
       'text_mood'],
      dtype='object')

## Cleaning data

In [24]:
data['url'] = 'https://www.vk.com/id' + data.index.astype(str)

NB: 1st gender - female, 2nd - male

In [28]:
data.is_nsfw

uid
169          1
204          0
276          0
586          0
1338         0
            ..
585709146    0
586222067    0
587578107    0
587988016    0
588191942    0
Name: is_nsfw, Length: 140711, dtype: int64

In [27]:
data[(data.gender==2) & (data.is_nsfw==1)].url.iloc[:100]

uid
19959        https://www.vk.com/id19959
29009        https://www.vk.com/id29009
37057        https://www.vk.com/id37057
47503        https://www.vk.com/id47503
90423        https://www.vk.com/id90423
                       ...             
5699822    https://www.vk.com/id5699822
5726541    https://www.vk.com/id5726541
5821226    https://www.vk.com/id5821226
5923204    https://www.vk.com/id5923204
6049084    https://www.vk.com/id6049084
Name: url, Length: 100, dtype: object

In [33]:
data[data.followers_qty<data.friends_qty].shape[0]/data.shape[0]

0.06189281577133273

In [93]:
data.shape

(140711, 15)

In [100]:
del data['url']

In [106]:
data.head()

,gender,followers_qty,friends_qty,pagerank_score,eigencentrality_score,hits_in,hits_out,flw_hits_in,flw_hits_out,is_nsfw,has_smile,number_of_faces,has_face,text_mood
uid,,,,,,,,,,,,,,
169,1.0,8621.0,0.0,0.000081,9.750758e-08,6.380952e-09,6.380963e-09,0.000000e+00,5.985958e-08,1,0,1,1,-0.142857
204,1.0,2912.0,240.0,0.000008,2.934387e-09,1.920279e-10,1.920282e-10,1.393409e-10,3.333800e-09,0,1,1,1,0.000000
276,1.0,3023.0,324.0,0.000008,3.314272e-11,2.168891e-12,2.168910e-12,0.000000e+00,1.877361e-54,0,1,1,1,-0.136364
586,2.0,0.0,0.0,0.000015,5.480418e-12,3.586429e-13,3.586450e-13,9.515891e-13,0.000000e+00,0,0,0,0,0.200000
1338,1.0,2878.0,0.0,0.000028,1.026794e-09,6.719400e-11,6.719414e-11,0.000000e+00,7.685454e-09,0,0,0,0,0.195876


In [111]:
data = data.dropna(how='all', subset=['gender', 'followers_qty', 'friends_qty', 'pagerank_score', 'eigencentrality_score', 
                                      'hits_in', 'hits_out', 'flw_hits_in', 'flw_hits_out']).copy(deep=True)

In [112]:
data.shape

(44637, 14)

In [113]:
data['url'] = 'https://www.vk.com/id' + data.index.astype(str)

In [92]:
data[(data.followers_qty<data.friends_qty) & (data.friends_qty>1000) & (data.is_nsfw==1) & (data.has_smile==0)].\
to_csv('./bots.nsfw.csv')

In [115]:
leaders = pd.read_csv('../data/leaders.labelled.csv')
leaders.head()

,Unnamed: 0,ID(alias),ID,Тип лидерства,комментарий
0,0,miller3,6182154,транзакционный,"немного трансформационный художник, комменты у..."
1,1,es_borisenko,61756961,служитель,"немного трансформационный, фотограф женщина"
2,2,ekaterina.kuzovkova,301218,транзакционный,"модель, только свои фото, комменты удалена опция"
3,3,id59772761,59772761,трансформационный,"диджей из Белгорода, может трансформационный, ..."
4,4,drevid_art,86539724,транзакционный,"парень, фотограф-дизайнер, любит себя"


In [116]:
leaders.shape

(99, 5)

In [123]:
leaders_in_our_data = data.loc[leaders.ID.to_list()].dropna().sort_values('followers_qty', ascending=False).copy(deep=True)

/home/kim/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [126]:
leaders_in_our_data['ID'] = leaders_in_our_data.index.astype(str)

In [128]:
leaders['ID'] = leaders.ID.astype(str)

In [130]:
leaders_in_our_data = leaders_in_our_data.merge(leaders).copy(deep=True)

In [131]:
del leaders_in_our_data['Unnamed: 0']

In [132]:
leaders_in_our_data.to_csv('./leaders_in_our_data.csv')

In [144]:
data = data[~((data.friends_qty>5000) & (data.followers_qty<1000))]

______

In [145]:
rating_columns = data.columns.to_list()
rating_columns.remove('is_nsfw')
rating_columns.remove('gender')
rating_columns.remove('has_smile')
rating_columns.remove('has_face')


ratings = {}
for c in rating_columns:
    ratings[c] = getRank(data, c)
    
ratings = pd.DataFrame(ratings)

In [146]:
ratings['url'] = ratings.index.map(lambda x : f'https://vk.com/id{x}')

In [147]:
resulting_df = pd.concat([ratings, data[['is_nsfw', 'gender', 'has_smile', 'has_face']]], axis=1)

In [148]:
resulting_df

,followers_qty,friends_qty,pagerank_score,eigencentrality_score,hits_in,hits_out,flw_hits_in,flw_hits_out,number_of_faces,text_mood,url,is_nsfw,gender,has_smile,has_face
169,74,10498,1596,817,817,817,8075,348,3449,36281,https://vk.com/id169,1,1.0,0,1
204,322,6264,30314,6781,6756,6756,2480,1451,3450,9633,https://vk.com/id204,0,1.0,1,1
276,312,5066,29428,21813,21813,21813,8076,7033,3451,36244,https://vk.com/id276,0,1.0,1,1
586,20773,10499,17519,26424,26401,26401,4582,8000,30758,6859,https://vk.com/id586,0,2.0,0,0
1338,327,10500,8421,9030,9030,9030,8077,686,30759,7159,https://vk.com/id1338,0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544507202,19898,9769,26325,15211,15238,15238,44509,44507,44509,3774,https://vk.com/id544507202,0,2.0,0,0
545345728,44509,44508,19031,9649,9783,9783,44510,44508,44510,34473,https://vk.com/id545345728,0,2.0,0,0
546453044,44510,44509,21063,14599,14684,14684,187,44509,44511,34474,https://vk.com/id546453044,0,2.0,0,0
546770024,20108,44510,44511,44457,44511,44511,44511,44510,30756,40361,https://vk.com/id546770024,0,2.0,1,1


In [149]:
top_10000_friendship_pagerank = set(resulting_df.sort_values('pagerank_score').index.to_list()[:10000])
top_10000_eigen = set(resulting_df.sort_values('eigencentrality_score').index.to_list()[:10000])
top_10000_hitsin = set(resulting_df.sort_values('hits_in').index.to_list()[:10000])
top_10000_hitsout = set(resulting_df.sort_values('hits_out').index.to_list()[:10000])
top_10000_flw_hitsin = set(resulting_df.sort_values('flw_hits_in').index.to_list()[:10000])
top_10000_flw_hitsout = set(resulting_df.sort_values('flw_hits_out').index.to_list()[:10000])
top_10000_followers = set(resulting_df.sort_values('followers_qty').index.to_list()[:10000])
top_10000_friends = set(resulting_df.sort_values('friends_qty').index.to_list()[:10000])

In [150]:
total_top = set.intersection(top_10000_friendship_pagerank, top_10000_eigen, top_10000_hitsin, top_10000_hitsout, 
                             top_10000_flw_hitsin, top_10000_flw_hitsout, top_10000_followers, top_10000_friends)

In [151]:
resulting_df['sum'] = resulting_df[resulting_df.columns.to_list()[:-7]].sum(axis=1)

In [80]:
resulting_df.loc[leaders.ID.astype(int).to_list()].dropna().sort_values('sum')

,followers_qty,friends_qty,pagerank_score,eigencentrality_score,hits_in,hits_out,flw_hits_in,flw_hits_out,number_of_faces,text_mood,url,is_nsfw,gender,has_smile,has_face,sum
936157,61,125,353,829,829,829,1859,3835,7027,80038,https://vk.com/id936157,0,1.0,1,1,8720
275542564,2918,242,2155,944,944,944,246,8005,28912,70820,https://vk.com/id275542564,0,1.0,1,1,16398
14831271,691,225,2858,4332,4332,4332,970,1332,36960,71520,https://vk.com/id14831271,0,2.0,0,0,19072
15464543,294,157,686,5428,5428,5428,1404,1788,16827,79967,https://vk.com/id15464543,0,1.0,1,1,20613
57536247,2856,237,1812,4323,4323,4323,1384,3987,21503,69919,https://vk.com/id57536247,0,2.0,1,1,23245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264740838,7765,41545,42839,42785,42832,42832,41941,41086,28754,74821,https://vk.com/id264740838,0,2.0,1,1,303625
437668940,44157,43969,44276,44222,44275,44275,521,43877,30464,106035,https://vk.com/id437668940,0,1.0,0,1,309572
377444384,7667,43362,43900,43846,43897,43897,43575,43162,30039,80046,https://vk.com/id377444384,0,1.0,0,1,313306
450721607,5564,44095,44346,44292,44345,44345,44212,44022,3445,74828,https://vk.com/id450721607,0,1.0,0,1,315221


In [170]:
top_1000_sum = set(resulting_df.sort_values('sum').index.to_list()[:10000])

In [171]:
resulting_df.loc[top_1000_sum].sort_values('followers_qty').url

1676192        https://vk.com/id1676192
54396750      https://vk.com/id54396750
4617              https://vk.com/id4617
38264770      https://vk.com/id38264770
398219          https://vk.com/id398219
                        ...            
92038265      https://vk.com/id92038265
124681665    https://vk.com/id124681665
301043192    https://vk.com/id301043192
340924441    https://vk.com/id340924441
381591773    https://vk.com/id381591773
Name: url, Length: 10000, dtype: object

In [172]:
sum(resulting_df.loc[top_1000_sum].loc[leaders.ID.astype(int).to_list()].sort_values('followers_qty').isna().all(axis=1))

/home/kim/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


55

In [176]:
resulting_df.loc[top_1000_sum].loc[leaders.ID.astype(int).to_list()].sort_values('followers_qty').dropna().url

/home/kim/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


38264770      https://vk.com/id38264770
241857          https://vk.com/id241857
12549495      https://vk.com/id12549495
267326387    https://vk.com/id267326387
1516042        https://vk.com/id1516042
2429322        https://vk.com/id2429322
4602035        https://vk.com/id4602035
936157          https://vk.com/id936157
978265          https://vk.com/id978265
12329839      https://vk.com/id12329839
35208371      https://vk.com/id35208371
20228219      https://vk.com/id20228219
5757882        https://vk.com/id5757882
43287096      https://vk.com/id43287096
153460605    https://vk.com/id153460605
37264961      https://vk.com/id37264961
21453059      https://vk.com/id21453059
305588155    https://vk.com/id305588155
3499511        https://vk.com/id3499511
15464543      https://vk.com/id15464543
34573330      https://vk.com/id34573330
61756961      https://vk.com/id61756961
17786179      https://vk.com/id17786179
14831271      https://vk.com/id14831271
1128374        https://vk.com/id1128374


In [68]:
resulting_df.loc[top_1000_sum].sort_values(by='sum').to_csv('./leadership.metrics-top-sum-all-scores.csv', index_label='vk_id')